# Homework 3

## FINM 37400 - 2025

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from dateutil.relativedelta import relativedelta
import statsmodels.api as sm
from scipy.optimize import minimize

***

# 1 HBS Case: Fixed-Income Arbitrage in a Financial Crisis (C): Spread and Swap Spread in November 2008

## Simplification of the setup

The date is Nov 4, 2008.

**Treasury bond**
* Suppose the Treasury bond matures exactly 30 years later, on Nov 4, 2038 rather than May 15, 2008. 
* The YTM of this freshly issued treasury is 4.193\% with a semiannual coupon of 4.50\%, same as is given in the case. (So we're just changing the maturity date to simplify things, but keeping the market data.)

**Swap**
* The fixed leg of the swap pays semiannually, with swap rate of 4.2560\%, as given in the case.
* The floating leg of the swap also pays semiannually--not quarterly--such that the payment dates are identical on both legs. Thus, it also resets the floating rate semiannually, not quarterly.
* The floating rate of the swap equals the repo rate used in the trade. Thus, these two rates cancel in the financing of the trade. (No need to consider the TED spread.) 

## Case Clarifications


### Duration Quotes
Bond
* Quote: Val01 of bond is .1746 per bp per $1 face value
* Class terminology: Modified dollar duration is .1746 per $100 face value

Swap
* Quote: DV01 of swap is 1.7mm per 1 billion notional.
* Class terminology: Modified dollar duration is 100(1.7/1000) per $100 face value.

Thus, modified dollar duration for each per 100 face is
* Bond = .1746
* Swap = .1700

### Hedge Ratio

In figuring out the hedge ratio, they set up the hedge per dollar of face value. 

    *so Mills would need to buy face amount $0.97 billion*
    
No, this hedge should be for market value, not face amount given that the case is already using **modified** duration which includes the dirty price.
    

### Maturity Mismatch

The maturity of the bond is August 2038, whereas the date is Nov 2008. Thus, the bond has less than 30 years to maturity, yet he is entering a 30-year swap. 

For simplicity, we imagine the bond is issued in Nov 2008 and thus has maturity of 30 years at the time of the case.

However, then the case quotes for the Nov price and YTM of the bond no longer are accurate. Use one and adjust the other. Namely, we could...
    * use the Nov 4 **YTM** quoted in the case, and re-adjust the the bond.
    * use the Nov 4 **price** quoted in the case, and re-adjust the YTM.
    
We do the former, keep the quoted YTM, assume time-to-maturity of `30`, and recalculate the Nov 2008 price. (It is close to the quoted price in the case.)

***

## 1.0.

Report the price of the 30-year T-bond in Nov 2008. Given the discussion about `Maturity Mismatch`, we are repricing the bond, so it will not exactly equal `105` as reported in the case.

In [80]:
ytm = 4.193
cpn = 4.5
yr = 30

def bond_price(ytm, cpn, yr):
    res_price = 0
    yield_denom = 1 + ytm / 200
    for i in range(1, 1 + yr * 2):
        cpn_val = cpn / (2 * (yield_denom ** i))
        res_price += cpn_val

    res_price += 100 / (yield_denom ** 60)

    return res_price




In [81]:
p1 = bond_price(ytm, cpn, yr)

print(f'The price is {round(p1, 2)}.')

The price is 105.21.


## 1.1

List the projected cashflows on May 4, 2009, exactly six months into the trade, on the first coupon and swap date.

#### Clarification
List these cashflows for face value of $1B, not the $0.97B noted in the case. As mentioned in "Case Clarifications", we will not use this number. Rather, we calculate our own hedge ratio in a problem below.

In [82]:
bond_fv = 1_000_000_000
bond_cf = bond_fv * cpn / 200
swap_rate = 4.256

# per 100 figures
bond_mod_ddur = 0.1746
swap_mod_ddur = 0.17


swap_fv = bond_fv

# repo and floating rate cancel out

swap_cf  = swap_fv * swap_rate / 200

print(f'Every 6 months, the bond pays {bond_cf} and you pay {swap_cf} for the swap.')

Every 6 months, the bond pays 22500000.0 and you pay 21280000.0 for the swap.


## 1.2

What is the duration of...
* the T-bond
* the swap

Remember that...
* the swap can be decomposed into a fixed-rate bond and a floating-rate note
* a floating-rate note has duration equal to the time until the next reset. Thus, at initialization, it has duration equal to 0.5 years.

Is the duration for the "paying-fixed" swap positive or negative? Is it bigger or smaller in magnitude than the T-bond?

For this problem, calculate the Macauley duration and the dollar (Macauley) duration.

In [83]:
def mc_duration(ytm, cpn, years_left, per_y=2, fixed=True, face=100):
    """
    Calculate the Macaulay Duration and Dollar Duration of a bond.

    Parameters:
    ytm (float): Yield to maturity (as a decimal).
    cpn (float): Annual coupon rate (as a decimal).
    years_left (int/float): Years remaining until maturity.
    per_y (int): Payments per year (default is 2 for semi-annual).
    fixed (bool): Whether it's a fixed coupon bond (default=True).
    face (float): Face value of the bond (default=100).

    Returns:
    tuple: (Macaulay Duration, Dollar Duration)
    """
    n = int(years_left * per_y)
    
    ytm_per_period = ytm / per_y / 100
    cpn_payment = (cpn * face) / per_y / 100

    time_periods = np.arange(1, n + 1)
    discounted_cf = cpn_payment / (1 + ytm_per_period) ** time_periods
    discounted_face = face / (1 + ytm_per_period) ** n
    
    # Bond price
    price = np.sum(discounted_cf) + discounted_face
    
    # Compute Macaulay Duration
    weighted_times = time_periods * discounted_cf
    mac_duration = (np.sum(weighted_times) + (n * discounted_face)) / (price * per_y)
    
    # Compute Modified Duration
    mod_duration = mac_duration / (1 + ytm_per_period)
    
    # Compute Dollar Duration
    dollar_duration = mod_duration * (price / 100) * face
    
    return mac_duration, dollar_duration

In [84]:
# t-bond

bond_mac, bond_ddur = mc_duration(ytm, cpn, 30, face=100)

print(bond_mac)
print(bond_ddur)

17.083633069693104
1760.5158840204253


In [85]:
swap_mac, swap_ddur = mc_duration(ytm, swap_rate, 30, face=100)

print(swap_mac)
print(swap_ddur)

17.283179937959357
1710.938179603965


# 1.2 Solution

The duration of the paying fixed swap is negative if you are paying fixed. It is smaller than the T-bond because of the SOFR adjustment. 

## 1.3

What hedge ratio should be used to balance the notional size of the Treasury bond with the notional size of the swap, such that it is a duration-neutral position?

Specifically, if the trader enters the swap paying fixed on \$500 million notional, how large of a position should they take in the Treasury bond?

In [86]:
swap_pos_fv = 500_000_000

bond_pos_fv = swap_pos_fv * (swap_ddur - 0.5) / (bond_ddur - 0.5) # 0.5 is for floating and repo respectively

print(bond_pos_fv)

485915551.99399406


## 1.4

Suppose it is May 4, 2009, exactly six months after putting the trade on.

The spread is at -28 bps due to...
* The YTM on a new 30-year bond has risen to 4.36\%
* The swap rate on a new 30-year swap has dropped to 4.08\%

Explain conceptually how this movement impacts the components of the trade.

# 1.4 solution

Leg 1: T-bond
Position loses value here because the original ytm was 4.193%. A rise to 4.36% would lose money.

Leg2: Swap
Swap paying 4.256% would lose value because you are now paying more than market rate. 

Both positions lose money when the spread goes to -28bps.

## 1.5

Calculate the value of the position on May 4, 2009, immediately after the first coupon and swap payments and swap reset. 

* Calculate the revised price of the Treasury bond by assuming you can apply the (May 4) 30-year YTM as a discount rate to the 29.5 year bond. (We are just using this for a rough approximation. You know that good pricing would require a discount curve, but let's not get bogged down with that here.)


* Calculate the value of the swap by decomposing it into a fixed-rate bond and a floating-rate bond.
    * The 29.5 year fixed-rate leg is priced using the (May 4) 30-year swap rate as a discount rate.
    * The floating-rate leg is priced at par given that floating-rate notes are par immediately after resets.
    
**Note**

You are being asked to calculate these valuations using the exact formula between price, cashflows, and YTM discount rate. We are not simply approximating with duration, as we already know the position was set up with zero dollar duration.

From the Discussion notebook, we have this formula expressing a bond's price as a function of the coupon, $c$, and the YTM, $y_j$.

$\begin{align*}
P_j(t,T,c) = \sum_{i=1}^{n-1}\frac{100\left(\frac{c}{2}\right)}{\left(1+\frac{y_j}{2}\right)^{2(T_i-t)}} + \frac{100\left(1+\frac{c}{2}\right)}{\left(1+\frac{y_j}{2}\right)^{2(T-t)}}
\end{align*}
$

In [87]:
def bond_price(cpn, ytm, years, years_past, compounding=2, FV=100):
    price = 0

    for i in np.arange(years_past + 1 / compounding, years + 1 / compounding, 1 / compounding):
        z = 1 / (1 + ytm/compounding) ** (compounding * (i - years_past))
        val = cpn * FV * z / compounding
        price += val

    price += (FV) / ((1 + ytm/compounding) ** (compounding * (years - years_past)))

    return float(price)

In [88]:
print(ytm)

bond_price_new = bond_price(cpn/100, 4.36/100, 30, 0.5)

print(bond_price_new)

4.193
102.31140068727244


## 1.6

Accounting for the change in value of the positions, as well as the 6-month cashflows paid on May 4, 
* what is the net profit and loss (pnl) of the position?
* what is the return on the equity capital, considering that there was a 2\% haircut (equity contribution) on the size of the initial treasury bond position.

***

# 2. Factor Duration

In [89]:
def regression_stats(df1, y_df, ann=1, constant=True):
    '''
        df1: x vars
        y_df: y vars
        ann: annualization constant (how many periods in a year)
    '''
    results = []  # To store results for each y variable

    for y_col in y_df.columns:
        y = y_df[y_col]

        # Add constant if needed
        if constant:
            df1_with_const = sm.add_constant(df1)
            column_names = ['Intercept'] + list(df1.columns)
        else:
            df1_with_const = df1
            column_names = list(df1.columns)

        # Fit the model
        model = sm.OLS(y, df1_with_const, missing = 'drop').fit()

        # Extract intercept and coefficients
        if constant:
            a = model.params[0] * ann  # Annualized intercept
            b = model.params[1:]      # Coefficients for other features
        else:
            a = 0                     # No intercept
            b = model.params[:]       # Use all coefficients

        # Calculate additional stats
        r_2 = model.rsquared
        e = model.resid
        sig_e = float(np.std(e))

        # Prepare a single row of results
        row = {"Y Variable": y_col, "R-squared": r_2, "Residual Std. Dev.": sig_e}

        if constant:
            row["Intercept"] = a
            row.update({name: coef for name, coef in zip(column_names[1:], b)})
        else:
            row.update({name: coef for name, coef in zip(column_names, b)})

        # Append the row to the results
        results.append(row)

    # Convert results to a DataFrame
    result_df = pd.DataFrame(results).set_index("Y Variable")
    return result_df

def regression_analysis(df1, y_df, ann=1, constant=True):
    df = regression_stats(df1, y_df, ann=ann, constant=constant)
    mean = y_df.mean()
    x_var = list(df1.columns)

    new_col = []

    for i in x_var:
        name = f'{i} Treynor Ratio'
        df[name] = mean / df[i] * ann

    df["Information Ratio"] = df["Intercept"]/df["Residual Std. Dev."] * np.sqrt(ann)

    return df

#The input is a list of different regression dataframes
def compare_regressions(lst):
    df1 = pd.concat(lst)

    return df1

### Data

This problem uses data from,
* `/data/yields.xlsx`
* `/data/treasury_ts_duration_2024-10-31.xlsx`

#### Load Yields

In [90]:
filepath = '../data/yields.xlsx'
yields = pd.read_excel(filepath, sheet_name='yields')
yields.set_index('caldt',inplace=True)

#### Load Prices and Durations of Two Treasuries

In [91]:
QUOTE_DATE = '2024-10-31'
filepath = f'../data/treasury_ts_duration_{QUOTE_DATE}.xlsx'

data = pd.read_excel(filepath,sheet_name='database')
data_info =  data.drop_duplicates(subset='KYTREASNO', keep='first').set_index('KYTREASNO')
data_info[['type','issue date','maturity date','cpn rate']]

,type,issue date,maturity date,cpn rate
KYTREASNO,,,,
207391,note,2019-08-15,2029-08-15,1.625
207392,bond,2019-08-15,2049-08-15,2.250


You will largely focus on the sheets which give the timeseries of prices and durations for each of the two securities, as shown in the following code.

In [92]:
SHEET_PRICE = 'price'
SHEET_DURATION = 'duration'
INDEX_NAME = 'quote date'

price = pd.read_excel(filepath,sheet_name=SHEET_PRICE).set_index(INDEX_NAME)
duration = pd.read_excel(filepath,sheet_name=SHEET_DURATION).set_index(INDEX_NAME)

### 2.1.

Construct the following yield-curve factors from the `yields` data set:

$\begin{align}
x^{\text{level}}_t =& \frac{1}{N_{\text{yields}}}\sum_{i=1}^{N_{\text{yields}}} y^{(i)}_t\\
x^{\text{slope}}_t =& y^{(30)}_t - y^{(1)}_t\\
x^{\text{curvature}}_t =& -y^{(1)}_t + 2 y^{(10)}_t - y^{(30)}_t
\end{align}$

In [93]:
y_path = '../data/yields.xlsx'
yields = pd.read_excel(y_path).set_index('caldt')

yields['level'] = yields.mean(axis=1)
yields['slope'] = yields[30] - yields[1]
yields['curvature'] = 2 * yields[10] - yields[1] - yields[30]

yields.index = pd.to_datetime(yields.index)


### 2.2

Get the bond prices and durations for the two bonds in the data set referenced above.

#### Align the data

Align the bond pricing data with the yield factor data, so that you have data for both in the intersection of their dates.


#### Estimate the regression

Estimate the regression in the form of day-over-day differences for both bond prices and factors. That is, we are using regression to approximate the factor duration equation,

$\begin{align}
\frac{dP}{P} = \alpha + \beta_L dx_{\text{level}} + \beta_S dx_{\text{slope}} + \beta_C dx_{\text{curvature}} + \epsilon
\end{align}$

Report the betas for each of these factors, for each of the bond prices.

In [94]:
issue_date = data_info.iloc[0]['issue date']


price = price.rename(columns={207391:'207391 price', 207392:'207392 price'})
duration = duration.rename(columns={207391:'207391 duration', 207392:'207392 duration'})



In [95]:
pd_df = pd.merge(price, duration, left_index=True, right_index=True)

comb_df = pd.merge(yields, pd_df, left_index=True, right_index=True)



In [97]:
X = comb_df[['level', 'slope', 'curvature']].diff()
Y = comb_df[['207391 price', '207392 price']].pct_change()

reg = regression_stats(X, Y)

reg

C:\Users\vince\AppData\Local\Temp\ipykernel_18948\4147967451.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  a = model.params[0] * ann  # Annualized intercept
C:\Users\vince\AppData\Local\Temp\ipykernel_18948\4147967451.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  a = model.params[0] * ann  # Annualized intercept


,R-squared,Residual Std. Dev.,Intercept,level,slope,curvature
Y Variable,,,,,,
207391 price,0.941657,0.001077,0.000064,-0.069938,-0.004759,-0.010926
207392 price,0.960223,0.002369,0.000069,-0.197983,-0.126491,0.074870


### 2.3.

Compare the "level" factor beta for each of the two treasuries with the average  duration for each bond as reported in the data set.

* How closely does the average duration for a bond compare to its "level" beta?
* What do you conclude about the usefulness of mathematical duration vs regression sensitivities?

In [98]:
print(comb_df['207391 duration'].mean())

print(comb_df['207392 duration'].mean())

display(reg['level'])


6.923664422237872
19.903216098452884


Y Variable
207391 price   -0.069938
207392 price   -0.197983
Name: level, dtype: float64

# 2.3 Solution

The avg duration for the bonds is very similar to the level beta, scaled by face value of 100 and negative. This makes sense because -duration measures the sensitivity to change in yield. The change in level is similar to the change in yield. Thus, it would make sense because we scaled the duration by 100 to get the Modified duration compared to the MC duration. The mathematical duration is very similar to the empirical duration we see.

### 2.4.

In the duration-hedged trade of `Homework 2, Section 2`, was the that trade was long or short this slope factor? 

Do you think the slope factor exposure had a large impact on the trade?

No new analysis needed, just draw a conclusion from the estimates above along with the trade construction in `HW 2, Sec 2`.

***

# 3 Calculating Duration Via Individual Cashflows

## *Optional, not submitted*

Use the data file `../data/treasury_quotes_2024-10-31.xlsx`.

### 3.1 
Set up the cashflow matrix. 

### 3.2
Extract the Nelson-Siegel spot discount curve, as you did in `Homework 1`.

### 3.3
For each treasury issue, calculate the duration as the weighted average of the (discounted!) cashflow maturity.

Report the summary statistics of the durations. (Use `.describe()` from pandas.)

### 3.4
How close are your duration estimates to the imputed durations given in the data source?

Report the summary statistics of the imputed durations minus your calculated durations from above.

### 3.5
Continue using your assumed discount rates of `4.5`\% to calculate the convexity of each issue.

Report the summary statistics of these convexity calculations.

***